## Step 3-4, Iteration 3 resubmission, MSAS before using Azure Machine Learning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('loanITERATION3.csv', encoding = "ISO-8859-1", low_memory=False)

## 3.1 Select the Data

In [3]:
col_drop = ['inq_fi', 'total_cu_tl', 'inq_last_12m', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il'] 
df = df.drop(col_drop, axis = 1)
col_drop = ['il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'annual_inc_joint']
df = df.drop(col_drop, axis = 1)
col_drop = ['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog']
df = df.drop(col_drop, axis = 1)
col_drop = ['policy_code', 'initial_list_status', 'zip_code', 'id', 'member_id', 'pymnt_plan','earliest_cr_line']
df = df.drop(col_drop, axis = 1)

In [4]:
df = df[(df['loan_status'] != 'Does not meet the credit policy. Status:Fully Paid') & (df['loan_status'] != 'Does not meet the credit policy. Status:Charged Off')]
# remove observations with loan_status == Current and Issued
df = df[(df['loan_status'] != 'Current') & (df['loan_status'] != 'Issued')]
print('No. of Observations: ', str(len(df)))
print('No. of Attributes: ' + str(len(df.columns)))

No. of Observations:  274390
No. of Attributes: 42


## 3.2 Clean the Data

### Missing values

In [5]:
# fill missing data with the median value (numeric) or the most frequent value (categorical)
for i in df:
    if df[i].isnull().sum() != 0:
        if df[i].dtype != 'object':
            fillmedian = df[i].median()
            df[i].fillna(fillmedian, inplace=True)
            print('{:<30} {:^6}'.format(i,str(fillmedian)))
        else:
            frequency = df[i].value_counts().index[0]
            df[i].fillna(frequency, inplace = True)
            print('{:<30} {:^6}'.format(i,frequency))

emp_length                     10 years
revol_util                      56.1 
last_pymnt_d                   15-Dec
last_credit_pull_d             16-Jan
collections_12_mths_ex_med      0.0  
tot_coll_amt                    0.0  
tot_cur_bal                    78674.0
total_rev_hi_lim               22010.0


## 3.3 Construct & Fomat the Data

In [6]:
# good and bad loan
df['target'] = 0
df.target = np.where(df.loan_status == 'Fully Paid', '1', '0')

In [7]:
df[['loan_status','target']].head()

,loan_status,target
0,Charged Off,0
1,Charged Off,0
2,Charged Off,0
3,Fully Paid,1
4,Fully Paid,1


# STEP 4 DATA TRANSFORMATION

## 4.1 Reduce the Data

In [8]:
col_drop = ['funded_amnt','funded_amnt_inv','total_pymnt_inv','out_prncp_inv']
df = df.drop(col_drop, axis=1)

In [9]:
col_drop = ['pub_rec','tot_coll_amt','collections_12_mths_ex_med','application_type','acc_now_delinq']
df = df.drop(col_drop, axis=1)
# drop loan status, keep target
df = df.drop('loan_status', axis=1)
print('No. of Observations: ', str(len(df)))
print('No. of Attributes: ' + str(len(df.columns)))

No. of Observations:  274390
No. of Attributes: 33


## 4.2 Project the Data

### Rename columns

In [10]:
df.rename(columns={'addr_state':'state'}, inplace=True)
df.rename(columns={'issue_d':'issue_date'}, inplace=True)
df.rename(columns={'dti':'monthly_payment_to_totaldebt'}, inplace=True)
df.rename(columns={'delinq_2yrs':'past_due_2years'}, inplace=True)
df.rename(columns={'inq_last_6mths':'inquery_6month'}, inplace=True)
df.rename(columns={'revol_util':'revol_utilization'}, inplace=True)
df.rename(columns={'open_rv_24m':'revol_amnt_24month'}, inplace=True)
df.rename(columns={'open_rv_12m':'revol_amnt_12month'}, inplace=True)
df.rename(columns={'total_rev_hi_lim':'total_credit_to_lim'}, inplace=True)

In [11]:
print(df.columns)

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_date', 'purpose', 'state', 'monthly_payment_to_totaldebt',
       'past_due_2years', 'inquery_6month', 'open_acc', 'revol_bal',
       'revol_utilization', 'total_acc', 'out_prncp', 'total_pymnt',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'last_credit_pull_d', 'tot_cur_bal', 'total_credit_to_lim', 'target'],
      dtype='object')


### Preprocessing the data

In [12]:
feature_name = df.drop('target', axis = 1).columns
x = df.drop('target', axis = 1)
y = df.target.values

In [13]:
# Encoding the fields
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical attributes
for i in x:
    if x[i].dtype == 'object':
        labelencoder_x = LabelEncoder()
        x[i] = labelencoder_x.fit_transform(x[i].astype(str))
        print('Label Encoded: ' + i)

Label Encoded: term
Label Encoded: grade
Label Encoded: sub_grade
Label Encoded: emp_length
Label Encoded: home_ownership
Label Encoded: verification_status
Label Encoded: issue_date
Label Encoded: purpose
Label Encoded: state
Label Encoded: last_pymnt_d
Label Encoded: last_credit_pull_d


### Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x = sc_x.fit_transform(x)


In [15]:
x

array([[-1.37134093,  1.80329344,  0.30395848, ..., -1.61822316,
        -0.33093969, -0.21912671],
       [-1.02021825,  1.80329344, -0.27807096, ..., -1.98124841,
        -0.33093969, -0.21912671],
       [-0.57749835, -0.55454092, -0.09759672, ..., -2.88881152,
        -0.33093969, -0.21912671],
       ..., 
       [-1.34386176, -0.55454092,  0.88824386, ...,  0.65068461,
         0.29296402, -0.84282033],
       [ 0.7170757 ,  1.80329344,  0.08738939, ...,  0.65068461,
         0.13197502,  7.89889409],
       [-0.35155854, -0.55454092,  1.19956194, ...,  0.65068461,
        -0.51550215, -0.41958591]])

In [17]:
x=pd.DataFrame(x)
y=pd.DataFrame(y)
spss_df = pd.concat([x,y],axis=1)
spss_df.to_excel('loan_azure.xlsx')

In [13]:
spss_df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,tot_cur_bal,total_credit_to_lim,target
0,2500,60 months,15.27,59.83,C,C4,0 year,RENT,30000.0,Source Verified,...,435.17,0.0,117.08,1.11,13-Apr,119.66,13-Sep,78674.0,22010.0,0
1,5375,60 months,12.69,121.45,B,B5,0 year,RENT,15000.0,Verified,...,533.42,0.0,269.29,2.52,12-Nov,121.45,13-Mar,78674.0,22010.0,0
2,9000,36 months,13.49,305.38,C,C1,0 year,RENT,30000.0,Source Verified,...,570.26,0.0,444.30,4.16,12-Jul,305.38,12-Nov,78674.0,22010.0,0
3,1000,36 months,16.29,35.31,D,D1,0 year,RENT,28000.0,Not Verified,...,270.17,0.0,0.00,0.00,15-Jan,36.32,16-Jan,78674.0,22010.0,1
4,8500,36 months,11.71,281.15,B,B3,0 year,RENT,25000.0,Source Verified,...,1622.21,0.0,0.00,0.00,14-Dec,581.59,15-Jan,78674.0,22010.0,1


In [14]:
print(len(df))
print(len(df.columns))

274390
33
